# Model Context Protocol (MCP)

## Courtesy and Acknowledgment

Thanks to Anthropic and the MCP team, we’ll be using a few code samples and examples from modelcontextprotocol.io.
Just a quick courtesy note — we’re truly grateful to the team for making this public.
Some of the code we’re using comes directly from the examples on their site, and we’ve also adapted ideas and structure from their documentation.



Prerequisite
Before we begin, make sure to download Claude Desktop from:
🔗 https://claude.ai/download

Refer the steps here https://modelcontextprotocol.io/quickstart/server

## Steps

First, we set up the environment

Then we build the server script

Next, we add the tools the model can call

And finally, we run the server

Once the server is up and running, the last step is to configure Claude Desktop to connect to the MCP server you just created.

You can repeat this for any number of MCP servers you want Claude to access — whether it’s for weather, finance, files, or anything else.

## Sample Server code

In [ ]:
##  weather.py

from typing import Any
import httpx
from mcp.server.fastmcp import FastMCP

# Initialize FastMCP server
mcp = FastMCP("weather")

# Constants
NWS_API_BASE = "https://api.weather.gov"
USER_AGENT = "weather-app/1.0"


async def make_nws_request(url: str) -> dict[str, Any] | None:
    """Make a request to the NWS API with proper error handling."""
    headers = {
        "User-Agent": USER_AGENT,
        "Accept": "application/geo+json"
    }
    async with httpx.AsyncClient() as client:
        try:
            response = await client.get(url, headers=headers, timeout=30.0)
            response.raise_for_status()
            return response.json()
        except Exception:
            return None

def format_alert(feature: dict) -> str:
    """Format an alert feature into a readable string."""
    props = feature["properties"]
    return f"""
Event: {props.get('event', 'Unknown')}
Area: {props.get('areaDesc', 'Unknown')}
Severity: {props.get('severity', 'Unknown')}
Description: {props.get('description', 'No description available')}
Instructions: {props.get('instruction', 'No specific instructions provided')}
"""


@mcp.tool()
async def get_alerts(state: str) -> str:
    """Get weather alerts for a US state.

    Args:
        state: Two-letter US state code (e.g. CA, NY)
    """
    url = f"{NWS_API_BASE}/alerts/active/area/{state}"
    data = await make_nws_request(url)

    if not data or "features" not in data:
        return "Unable to fetch alerts or no alerts found."

    if not data["features"]:
        return "No active alerts for this state."

    alerts = [format_alert(feature) for feature in data["features"]]
    return "\n---\n".join(alerts)

@mcp.tool()
async def get_forecast(latitude: float, longitude: float) -> str:
    """Get weather forecast for a location.

    Args:
        latitude: Latitude of the location
        longitude: Longitude of the location
    """
    # First get the forecast grid endpoint
    points_url = f"{NWS_API_BASE}/points/{latitude},{longitude}"
    points_data = await make_nws_request(points_url)

    if not points_data:
        return "Unable to fetch forecast data for this location."

    # Get the forecast URL from the points response
    forecast_url = points_data["properties"]["forecast"]
    forecast_data = await make_nws_request(forecast_url)

    if not forecast_data:
        return "Unable to fetch detailed forecast."

    # Format the periods into a readable forecast
    periods = forecast_data["properties"]["periods"]
    forecasts = []
    for period in periods[:5]:  # Only show next 5 periods
        forecast = f"""
{period['name']}:
Temperature: {period['temperature']}°{period['temperatureUnit']}
Wind: {period['windSpeed']} {period['windDirection']}
Forecast: {period['detailedForecast']}
"""
        forecasts.append(forecast)

    return "\n---\n".join(forecasts)


if __name__ == "__main__":
    # Initialize and run the server
    mcp.run(transport='stdio')


In [ ]:
## yahoo_finance.py

# yahoo_finance.py

from typing import Any
from mcp.server.fastmcp import FastMCP
import yfinance as yf
import asyncio

mcp = FastMCP("yahoo_finance")

STOCK_SYMBOLS = {
    "nvidia": "NVDA",
    "nvda": "NVDA",
    "apple": "AAPL",
    "aapl": "AAPL",
    "google": "GOOGL",
    "googl": "GOOGL",
    "microsoft": "MSFT",
    "msft": "MSFT",
}


def fetch_stock_price(symbol: str) -> float | None:
    try:
        stock = yf.Ticker(symbol)
        data = stock.history(period="1d")
        if not data.empty:
            return data["Close"].iloc[-1]
    except Exception:
        return None


@mcp.tool()
async def get_stock_price(company: str) -> str:
    company_lower = company.lower()
    symbol = STOCK_SYMBOLS.get(company_lower)
    if not symbol:
        for name, sym in STOCK_SYMBOLS.items():
            if name in company_lower:
                symbol = sym
                break

    if not symbol:
        return "Sorry, I couldn't identify the stock symbol."

    loop = asyncio.get_running_loop()
    price = await loop.run_in_executor(None, fetch_stock_price, symbol)

    if price is None:
        return f"Sorry, I couldn't retrieve the stock price for {company}."

    return f"The latest stock price for {company.title()} ({symbol}) is ${price:.2f}"


if __name__ == "__main__":
    mcp.run(transport="stdio")

